### Feature Engineering Pipeline must include:
- Replace variable Defaulted to 1, 0
- Replace variable NewCreditCustomer to 1, 0
- Drop variables ['ID', 'Status', 'CreditScoreEsEquifaxRisk','DefaultDate', 'EmploymentPosition','City']
- Create new binary variables for columns with NA
- Convert variables ['Education','EmploymentStatus','Gender','HomeOwnershipType','MaritalStatus', 'VerificationType'] from float to int
- Apply log to numerical variables to ensure normal distribution (eventually)
- Convert DateOfBirth to Age and extract month from LoanDate
- Convert strings to numbers

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

# to divide train and test set
from sklearn.model_selection import train_test_split

# feature scaling
from sklearn.preprocessing import MinMaxScaler

# to visualise al the columns in the dataframe
pd.pandas.set_option('display.max_columns', None)

# to ignore warnings
import warnings
warnings.simplefilter('ignore')

In [2]:
data = pd.read_csv(filepath_or_buffer=r'loans_data.csv')

print(data.shape)
data.head()

(27917, 32)


,ID,Status,Amount,ApplicationSignedHour,ApplicationSignedWeekday,City,Country,CreditScoreEsEquifaxRisk,DateOfBirth,DebtToIncome,Education,EmploymentDurationCurrentEmployer,EmploymentPosition,EmploymentStatus,ExistingLiabilities,Gender,HomeOwnershipType,IncomeFromPrincipalEmployer,IncomeTotal,Interest rate (APR),LoanDate,LoanDuration,MaritalStatus,NewCreditCustomer,NoOfPreviousLoansBeforeLoan,OccupationArea,UseOfLoan,VerificationType,WorkExperience,PreviousScore,Defaulted,DefaultDate
0,0,Current,5000.0,12,4,AESPA,EE,NaN,21/11/1975,51.52,4.0,UpTo5Years,Worker,3.0,9,0.0,6.0,1000.0,1000.0,24.52,05/05/2015,60,4.0,False,1,8.0,0,1.0,15To25Years,0.0957,False,NaN
1,2,Repaid,530.0,14,2,PÄRNU,EE,NaN,25/10/1969,30.32,4.0,MoreThan5Years,SpecialistOfficeWorker,3.0,7,1.0,9.0,633.0,633.0,25.68,19/10/2015,60,1.0,True,0,1.0,2,4.0,15To25Years,0.1034,False,NaN
2,3,Current,5500.0,22,4,TALLINN,EE,NaN,22/11/1965,29.57,4.0,MoreThan5Years,Owner,5.0,1,0.0,1.0,550.0,550.0,21.62,02/09/2015,60,4.0,True,0,13.0,2,3.0,MoreThan25Years,0.0772,False,NaN
3,4,Repaid,6900.0,15,3,KEHTNA,EE,NaN,05/12/1983,45.75,2.0,MoreThan5Years,Worker,3.0,10,0.0,1.0,833.0,833.0,21.63,20/05/2015,60,3.0,False,1,19.0,7,4.0,5To10Years,0.0773,False,NaN
4,5,Current,2655.0,10,3,KIVIÕLI,EE,NaN,13/07/1980,25.40,4.0,MoreThan5Years,Worker,3.0,6,1.0,4.0,341.0,341.0,26.88,29/12/2015,60,3.0,True,0,17.0,2,4.0,5To10Years,0.1898,False,NaN


### Separate into train and test sets

In [1]:
X_train, X_test, y_train, y_test = train_test_split(data, data.Defaulted, 
                                                   test_size = 0.2,
                                                   random_state = 0)
X_train.shape, X_test.shape

NameError: name 'train_test_split' is not defined

In [4]:
X_train['Defaulted'].replace([True,False],[1,0],inplace=True)
X_test['Defaulted'].replace([True,False],[1,0],inplace=True)

### Variables to drop

In [5]:
vars_list = ['ID', 'Status', 'CreditScoreEsEquifaxRisk','DefaultDate']
X_train.drop(vars_list, axis = 1, inplace = True)
X_test.drop(vars_list, axis = 1, inplace = True)

### Missing Values

#### Categorical Variables

In [6]:
cat_with_na = [var for var in X_train.columns if X_train[var].isnull().sum()>1 and X_train[var].dtypes=='O']

for var in cat_with_na:
    print(var, np.round(X_train[var].isnull().mean(), 3),  ' missing values')

City 0.007  missing values
EmploymentDurationCurrentEmployer 0.025  missing values
EmploymentPosition 0.014  missing values
WorkExperience 0.002  missing values


In [7]:
# replace the missing values
for var in cat_with_na:
    
    # calculate the mode
    mode_val = X_train[var].mode()[0]
    
    # train
    X_train[var+'_na'] = np.where(X_train[var].isnull(), 1, 0)
    X_train[var].fillna(mode_val, inplace=True)
    
    # test
    X_test[var+'_na'] = np.where(X_test[var].isnull(), 1, 0)
    X_test[var].fillna(mode_val, inplace=True)
    
# check that we have no more missing values in the engineered variables
X_train[cat_with_na].isnull().sum()

City                                 0
EmploymentDurationCurrentEmployer    0
EmploymentPosition                   0
WorkExperience                       0
dtype: int64

#### Continuos Variables

In [8]:
cont_with_na = [var for var in X_train.columns if X_train[var].isnull().sum()>1 and X_train[var].dtypes!='O']

for var in cont_with_na:
    print(var, np.round(X_train[var].isnull().mean(), 3),  ' missing values')

DebtToIncome 0.002  missing values
Education 0.002  missing values
EmploymentStatus 0.006  missing values
Gender 0.002  missing values
HomeOwnershipType 0.048  missing values
MaritalStatus 0.002  missing values
OccupationArea 0.003  missing values
VerificationType 0.002  missing values
PreviousScore 0.079  missing values


In [9]:
# replace the missing values
for var in cont_with_na:
    
    # calculate the mode
    mean_val = X_train[var].mean()
    
    # train
    X_train[var+'_na'] = np.where(X_train[var].isnull(), 1, 0)
    X_train[var].fillna(mean_val, inplace=True)
    
    # test
    X_test[var+'_na'] = np.where(X_test[var].isnull(), 1, 0)
    X_test[var].fillna(mean_val, inplace=True)
    
# check that we have no more missing values in the engineered variables
X_train[cont_with_na].isnull().sum()

DebtToIncome         0
Education            0
EmploymentStatus     0
Gender               0
HomeOwnershipType    0
MaritalStatus        0
OccupationArea       0
VerificationType     0
PreviousScore        0
dtype: int64

In [10]:
print([var for var in X_train.columns if X_train[var].isnull().sum()>0])
print([var for var in X_test.columns if X_test[var].isnull().sum()>0])

[]
[]


### Temporal Variables

In [11]:
def get_age(df, var):
    X = df.copy()
    X[var] = pd.to_datetime(X[var])
    X[var] = pd.DatetimeIndex(X[var]).year
    return X[var]

def get_month(df, var):
    X = df.copy()
    X[var] = pd.to_datetime(X[var])
    X[var] = pd.DatetimeIndex(X[var]).month
    return X[var]

In [12]:
temp_variables = ['DateOfBirth', 'LoanDate']

X_train[temp_variables[0]] = get_age(X_train, temp_variables[0])
X_test[temp_variables[0]] = get_age(X_test, temp_variables[0])

X_train[temp_variables[1]] = get_month(X_train, temp_variables[1])
X_test[temp_variables[1]] = get_month(X_test, temp_variables[1])

In [13]:
print([var for var in X_train.columns if X_train[var].isnull().sum()>0])
print([var for var in X_test.columns if X_test[var].isnull().sum()>0])

[]
[]


### Categorical Variables

Convert the features with high cardinality with featutes with few labels

In [14]:
EmploymentPosition_top10 = [x for x in X_train['EmploymentPosition'].value_counts().head(7).index]
City_top10 = [x for x in X_train['City'].value_counts().head(10).index]

cat_vars = [var for var in X_train.columns if X_train[var].dtypes == 'O']

In [15]:
def get_top_categories(df, var, top_vars):
    X = df.copy()
    X[var] = [x if x in top_vars else 'Other' for x in X[var]]
    return X[var]

for var, top_vars in zip(['EmploymentPosition', 'City'], [EmploymentPosition_top10,City_top10]):
    X_train[var] = get_top_categories(X_train, var, top_vars)
    X_test[var] = get_top_categories(X_test, var, top_vars)

Transformation of strings into numbers

In [16]:
def replace_categories(train, test, var, target):
    ordered_labels = train.groupby([var])[target].mean().sort_values().index
    ordinal_label = {k:i for i, k in enumerate(ordered_labels, 0)} 
    train[var] = train[var].map(ordinal_label)
    test[var] = test[var].map(ordinal_label)

In [17]:
for var in cat_vars:
    replace_categories(X_train, X_test, var, 'Defaulted')

### Feature Scaling

In [18]:
train_vars = [var for var in X_train.columns if var not in ['Defaulted']]
len(train_vars)

40

In [19]:
# fit scaler
scaler = MinMaxScaler() # create an instance
scaler.fit(X_train[train_vars]) #  fit  the scaler to the train set for later use

# transform the train and test set, and add on the Id and SalePrice variables
X_train = pd.concat([X_train[['Defaulted']].reset_index(drop=True),
                    pd.DataFrame(scaler.transform(X_train[train_vars]), columns=train_vars)],
                    axis=1)

X_test = pd.concat([X_test[['Defaulted']].reset_index(drop=True),
                    pd.DataFrame(scaler.transform(X_test[train_vars]), columns=train_vars)],
                    axis=1)

In [20]:
print([var for var in X_train.columns if X_train[var].isnull().sum()>0])
print([var for var in X_test.columns if X_test[var].isnull().sum()>0])
X_train.head()

[]
[]


,Defaulted,Amount,ApplicationSignedHour,ApplicationSignedWeekday,City,Country,DateOfBirth,DebtToIncome,Education,EmploymentDurationCurrentEmployer,EmploymentPosition,EmploymentStatus,ExistingLiabilities,Gender,HomeOwnershipType,IncomeFromPrincipalEmployer,IncomeTotal,Interest rate (APR),LoanDate,LoanDuration,MaritalStatus,NewCreditCustomer,NoOfPreviousLoansBeforeLoan,OccupationArea,UseOfLoan,VerificationType,WorkExperience,PreviousScore,City_na,EmploymentDurationCurrentEmployer_na,EmploymentPosition_na,WorkExperience_na,DebtToIncome_na,Education_na,EmploymentStatus_na,Gender_na,HomeOwnershipType_na,MaritalStatus_na,OccupationArea_na,VerificationType_na,PreviousScore_na
0,1,0.234263,0.695652,0.500000,0.5,0.000000,0.6750,0.145198,0.50,0.666667,0.142857,0.25,0.055556,0.5,0.111111,0.000000,0.012624,0.149441,0.636364,0.593220,0.50,1.0,0.0,0.45,0.054545,1.000000,0.4,0.281769,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0.238501,1.000000,0.000000,0.5,0.333333,0.4125,0.075163,0.75,0.333333,0.714286,0.25,0.027778,0.0,0.777778,0.014180,0.014180,0.109500,0.454545,1.000000,0.00,1.0,0.0,0.10,0.027273,0.666667,0.0,0.158786,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0.102402,0.652174,0.333333,0.5,0.000000,0.5750,0.344172,1.00,1.000000,0.857143,0.25,0.277778,0.5,0.333333,0.006466,0.008414,0.108263,0.636364,0.186441,0.25,1.0,0.0,0.85,0.000000,1.000000,0.8,0.434872,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0.799383,0.869565,0.333333,0.2,0.000000,0.7125,0.231616,0.75,0.500000,0.571429,0.25,0.027778,0.5,0.333333,0.013534,0.013534,0.118316,0.363636,1.000000,0.25,1.0,0.0,0.70,0.063636,0.000000,0.6,0.201637,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0.279943,0.695652,0.000000,0.5,0.000000,0.5000,0.394822,0.75,0.666667,0.714286,0.25,0.055556,0.5,0.111111,0.006015,0.006015,0.112555,0.000000,1.000000,0.00,1.0,0.0,0.10,0.018182,0.000000,0.6,0.340646,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
X_train.to_csv('xtrain.csv', index=False)
X_test.to_csv('xtest.csv', index=False)